In [3]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pandas as pd
import numpy as np
import scipy
import sklearn
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pandas import Series, DataFrame
import nltk

import spacy
import string

from gensim import corpora, models, similarities

import tensorflow as tf
import tensorflow_hub as hub

from gensim.models import Word2Vec

In [1]:
from google.colab import files 
uploaded = files.upload() 

Saving train.csv to train (1).csv


In [4]:
df = pd.read_csv('train.csv')
len(df)

711

In [5]:
df.dropna(subset=['label'], inplace=True)
len(df)

711

In [6]:
df['label']=df['label'].map(lambda x:int(x))
df['label'].value_counts()

2    237
1    237
0    237
Name: label, dtype: int64

In [7]:
df

,CONTENT,label
0,welcome to morningstar co uk -pron- have be re...,2
1,three value invest etf worth consider now offe...,2
2,the yield curve of u s government bond of diff...,2
3,adient plc adnt report adjusted earning per sh...,2
4,wilmington de accesswire september 16 2019 rig...,2
5,if -pron- want to compound wealth in the stock...,2
6,houston sept 12 2019 globe newswire the board ...,2
7,howard marks put -pron- nicely when -pron- say...,2
8,the woodlands texas sept 18 2019 globe newswir...,2
9,approval support by two phase 3 trial demonstr...,2


In [8]:
from google.colab import files 
uploaded = files.upload() 

Saving test.csv to test.csv


In [8]:
tdf = pd.read_csv('test.csv')
len(tdf)
tdf

,CONTENT,label
0,mclean va oct 08 2019 globe newswire gladstone...,1
1,add detail background share sept 9 reuters fre...,0
2,acceleron pharma inc xlrn announce the discont...,0
3,corona ca accesswire october 3 2019 adomani in...,0
4,corona ca accesswire september 20 2019 adomani...,2
5,corona ca accesswire september 26 2019 adomani...,2
6,bloomberg tilman fertitta have to wait to land...,1
7,plan to resume trade on over the counter marke...,1
8,beijing may 23 2019 globe newswire chinacache ...,0
9,beijing aug 28 2019 globe newswire chinacache ...,1


In [9]:
tdf.dropna(subset=['label'], inplace=True)
len(tdf)

370

In [0]:
x = df['CONTENT']
y = df['label']
x_test=tdf['CONTENT']
y_test=tdf['label']

In [11]:
import sklearn.model_selection
from sklearn.model_selection import train_test_split
SEED = 42
x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=.02, random_state=SEED)
print (x_train)

611    investor interested in business service stock ...
63     atlanta sept 18 2019 prnewswire follow the fed...
404    investor in celgene corp symbol celg see new o...
314    although wayfair inc nyse w seem well position...
333    while merck co and roche holding ag have take ...
465    wynyard uk may 14 2019 prnewswire venator mate...
208    community solar in minnesota allow business pu...
544    howard marks put -pron- nicely when -pron- say...
515    emerge market be outperform the u s economy th...
81     juul labs inc and other e cigarette maker woul...
677    new york oct 9 2019 prnewswire laminate floor ...
65     be the ability to time the market more of a da...
697    the late 13f reporting period have come and go...
291    merit medical systems inc nasdaq mmsi which be...
356    good afternoon from los angeles wherever -pron...
39     the lovesac company love come out with a quart...
148    cleveland cliffs inc clf will begin trade ex d...
284    new york sept 26 2019 gl

In [110]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence
import multiprocessing
from sklearn import utils

def labelize_news_ug(news,label):
    result = []
    prefix = label
    for i, t in zip(news.index, news):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result
  
all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v = labelize_news_ug(all_x, 'all')

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  if sys.path[0] == '':


In [111]:


cores = multiprocessing.cpu_count()
model_ug_dbow = Doc2Vec(dm=0, size=1000, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dbow.build_vocab([x for x in tqdm(all_x_w2v)])

for epoch in range(30):
    model_ug_dbow.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dbow.alpha -= 0.002
    model_ug_dbow.min_alpha = model_ug_dbow.alpha
    
def get_vectors(model, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = model.docvecs[prefix]
        n += 1
    return vecs
  
train_vecs_dbow = get_vectors(model_ug_dbow, x_train, 1000)
validation_vecs_dbow = get_vectors(model_ug_dbow, x_validation, 1000)

100%|██████████| 1081/1081 [00:00<00:00, 823323.52it/s]


In [0]:
test_vecs_dbow = get_vectors(model_ug_dbow, x_test, 1000)

In [113]:
cores = multiprocessing.cpu_count()
model_ug_dmm = Doc2Vec(dm=1, dm_mean=1, size=1000, window=4, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dmm.build_vocab([x for x in tqdm(all_x_w2v)])

for epoch in range(30):
    model_ug_dmm.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dmm.alpha -= 0.002
    model_ug_dmm.min_alpha = model_ug_dmm.alpha
    
train_vecs_dmm = get_vectors(model_ug_dmm, x_train, 1000)
validation_vecs_dmm = get_vectors(model_ug_dmm, x_validation, 1000)
test_vecs_dmm = get_vectors(model_ug_dmm, x_test, 1000)

100%|██████████| 1081/1081 [00:00<00:00, 439814.01it/s]


In [0]:
def get_concat_vectors(model1,model2, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = np.append(model1.docvecs[prefix],model2.docvecs[prefix])
        n += 1
    return vecs
train_vecs_dbow_dmm = get_concat_vectors(model_ug_dbow,model_ug_dmm, x_train, 2000)
validation_vecs_dbow_dmm = get_concat_vectors(model_ug_dbow,model_ug_dmm, x_validation, 2000)
test_vecs_dbow_dmm = get_concat_vectors(model_ug_dbow,model_ug_dmm, x_test, 2000)


In [115]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmm, y_train)

print clf.score(train_vecs_dbow_dmm, y_train)
print clf.score(validation_vecs_dbow_dmm, y_validation)

1.0
0.5333333333333333


In [0]:
from gensim.models.phrases import Phrases
from gensim.models.phrases import Phraser
tokenized_train = [t.split() for t in x_train]
phrases = Phrases(tokenized_train)
bigram = Phraser(phrases)
tg_phrases = Phrases(bigram[tokenized_train])
trigram = Phraser(tg_phrases)
def labelize_news_tg(news,label):
    result = []
    prefix = label
    for i, t in zip(news.index, news):
        result.append(LabeledSentence(trigram[bigram[t.split()]], [prefix + '_%s' % i]))
    return result

In [117]:
all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v_tg = labelize_tweets_tg(all_x, 'all')

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


In [118]:
cores = multiprocessing.cpu_count()
model_tg_dmm = Doc2Vec(dm=1, dm_mean=1, size=1000, window=4, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_tg_dmm.build_vocab([x for x in tqdm(all_x_w2v_tg)])

100%|██████████| 1081/1081 [00:00<00:00, 369944.73it/s]


In [0]:
train_vecs_dmm_tg = get_vectors(model_tg_dmm, x_train, 1000)
validation_vecs_dmm_tg = get_vectors(model_tg_dmm, x_validation, 1000)
test_vecs_dmm_tg = get_vectors(model_tg_dmm, x_test, 1000)

In [0]:
model_ug_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
model_tg_dmm.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
train_vecs_ugdbow_tgdmm = get_concat_vectors(model_ug_dbow,model_tg_dmm, x_train, 2000)
validation_vecs_ugdbow_tgdmm = get_concat_vectors(model_ug_dbow,model_tg_dmm, x_validation, 2000)
test_vecs_ugdbow_tgdmm = get_concat_vectors(model_ug_dbow,model_tg_dmm, x_test, 2000)

In [0]:
train_w2v_final = train_vecs_ugdbow_tgdmm
validation_w2v_final = validation_vecs_ugdbow_tgdmm
test_w2v_final = test_vecs_ugdbow_tgdmm

In [123]:
seed=42
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
filepath="w2v_01_best_weights.{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_acc', patience=5, mode='max') 
callbacks_list = [checkpoint, early_stop]
np.random.seed(seed)
model_w2v_01 = Sequential()
model_w2v_01.add(Dense(256, activation='relu', input_dim=2000))
model_w2v_01.add(Dense(256, activation='relu'))
model_w2v_01.add(Dense(256, activation='relu'))
model_w2v_01.add(Dense(1, activation='sigmoid'))
model_w2v_01.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_w2v_01.fit(train_w2v_final, y_train, validation_data=(validation_w2v_final, y_validation), 
                 epochs=100, batch_size=32, verbose=2, callbacks=callbacks_list)

Train on 696 samples, validate on 15 samples
Epoch 1/100
 - 1s - loss: -9.3296e-02 - acc: 0.3362 - val_loss: -4.0484e+00 - val_acc: 0.2000

Epoch 00001: val_acc improved from -inf to 0.20000, saving model to w2v_01_best_weights.01-0.2000.hdf5
Epoch 2/100
 - 0s - loss: -2.4473e+00 - acc: 0.3463 - val_loss: -6.4456e+00 - val_acc: 0.3333

Epoch 00002: val_acc improved from 0.20000 to 0.33333, saving model to w2v_01_best_weights.02-0.3333.hdf5
Epoch 3/100
 - 0s - loss: -3.9535e+00 - acc: 0.4885 - val_loss: -5.5211e+00 - val_acc: 0.4000

Epoch 00003: val_acc improved from 0.33333 to 0.40000, saving model to w2v_01_best_weights.03-0.4000.hdf5
Epoch 4/100
 - 0s - loss: -4.6843e+00 - acc: 0.5546 - val_loss: -5.2184e+00 - val_acc: 0.4000

Epoch 00004: val_acc did not improve from 0.40000
Epoch 5/100
 - 0s - loss: -4.8496e+00 - acc: 0.5920 - val_loss: -5.7232e+00 - val_acc: 0.4000

Epoch 00005: val_acc did not improve from 0.40000
Epoch 6/100
 - 0s - loss: -4.9612e+00 - acc: 0.6307 - val_loss: -

In [0]:
from keras.models import load_model
loaded_w2v_model = load_model('w2v_01_best_weights.07-0.4667.hdf5')

In [125]:
loaded_w2v_model.evaluate(x=validation_w2v_final, y=y_validation)

15/15 [==============================] - 0s 13ms/step


[-6.20440149307251, 0.46666666865348816]

In [128]:
loaded_w2v_model.evaluate(x=test_w2v_final, y=y_test)

370/370 [==============================] - 0s 109us/step


[2.9934623460511904, 0.37567567599786295]